**Land Ownership Cleaning**

In [ ]:
import geopandas as gpd
import sys
from pathlib import Path

notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root))


from scripts import config

Let's load and take a look at the data.

In [ ]:
gdf = gpd.read_file(config.RAW_DATA_PATH / "PADUS4_1_State_CO_GDB_KMZ" / "PADUS4_1_StateCO.gdb")
print("Columns:", gdf.columns.tolist())
print("State names: ", gdf['State_Nm'].unique())

Data is already isolated to CO — if we want other states in the future, we can link to the full U.S. dataset and select desired states.

Let's filter down to useful columns.

In [ ]:
columns_keep = [
    "Mang_Name",
    "Loc_Mang",
    "Unit_Nm",
    "Loc_Nm",
    "Des_Tp",
    "Pub_Access",
    "Own_Name",
    "State_Nm",
    'Shape_Length', 
    'Shape_Area',
    "geometry"
]

gdf_cols = gdf[columns_keep]
gdf_cols.head()


Now, we want to filter on relevant land managers. Even if a piece of land is owned by a relevant owner, management determines local camping legality, so we'll filter on manager name.

In [ ]:
print("Manager names: \n", gdf_cols['Mang_Name'].unique())

From this list, we want to keep BLM, USFS, NPS, and CNTY.

In [ ]:
managers_keep = [
    'BLM',
    'USFS',
    'NPS',
    'CNTY'
]

gdf_filtered = gdf_cols[gdf_cols['Mang_Name'].isin(managers_keep)]
gdf_filtered.head()

Let's make sure all geometries have the same type:

In [ ]:
print("Geometry types:", gdf_filtered.geom_type.unique())

All good. Let's project to the common CRS for future intersections.

In [ ]:
gdf_filtered = gdf_filtered.to_crs(config.BUFFER_CRS)

Finally, let's fix invalid shapes.

In [ ]:
# Identify invalid shapes
invalid = ~gdf_filtered.is_valid
print(f"Invalid geometries: {invalid.sum()}")

# Only fix those
gdf_filtered.loc[invalid, "geometry"] = gdf_filtered.loc[invalid, "geometry"].buffer(0)


Ready to export!

In [ ]:
output_folder = config.OUTPUT_DIR / "land_ownership_cleaned"
output_folder.mkdir(parents=True, exist_ok=True)

output_fp = output_folder / "land_ownership_cleaned.gpkg"
gdf_filtered.to_file(output_fp, driver="GPKG")

print("Cleaned land ownership data exported to: ", output_fp)